# Mission to Mars
Rutgers Data Science Bootcamp Week 13 Assignement

## Python Imports

In [2]:
import time
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
from splinter import Browser

## Step 1 - Scraping

### NASA Mars News

In [2]:
def get_mars_news():
    url = 'https://mars.nasa.gov/news/'
    with Browser('chrome') as browser:
        browser.visit(url)
        time.sleep(1)
        html = browser.html
        soup = bs4(html,'lxml')
        news_title = soup.find('div', class_='content_title').a.text.strip()
        news_p = soup.find('div', class_='article_teaser_body').text.strip()
    return (news_title, news_p)

In [5]:
get_mars_news()

('NASA’s First Mission to Study the Interior of Mars Awaits May 5 Launch',
 'All systems are go for NASA’s next launch to the Red Planet.')

### JPL Mars Space Images - Featured Image

In [70]:
def get_feature_image_url():
    base_url = 'https://www.jpl.nasa.gov/'
    url = f'{base_url}/spaceimages/?search=&category=Mars'
    with Browser('chrome') as browser:
        browser.visit(url)
        time.sleep(1)
        browser.click_link_by_id('full_image')
        time.sleep(1)
        browser.click_link_by_partial_text('more info')
        time.sleep(1)
        soup = bs4(browser.html, 'lxml')
        image_url = soup.find('figure', class_='lede').a['href']  
    
    return base_url + image_url

In [71]:
get_feature_image_url()

'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA18840_hires.jpg'

### Mars Weather

In [ ]:
def get_mars_weather():
    return bs4(
        requests.get(
            'https://twitter.com/marswxreport?lang=en'
        ).text, 'lxml').find('p', class_='TweetTextSize').text    

In [73]:
get_mars_weather()

'Sol 2039 (May 02, 2018), Sunny, high 0C/32F, low -74C/-101F, pressure at 7.28 hPa, daylight 05:23-17:20'

### Mars Facts
Ref: https://pythonprogramminglanguage.com/web-scraping-with-pandas-and-beautifulsoup/

In [7]:
def get_mars_fact_html():
    return pd.read_html(str(
        bs4(
        requests.get(
            'https://space-facts.com/mars/'
        ).text, 'lxml').find_all('table', class_='tablepress-id-mars')
    ))[0].rename(columns={0: 'Measure', 1: 'Value'}).to_html(index=False)

In [8]:
get_mars_fact_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Measure</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

### Mars Hemisperes

In [76]:
def get_mars_hemispheres():
    base_url = 'https://astrogeology.usgs.gov'
    url = f'{base_url}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    hemisphere_image_urls = []
    with Browser('chrome') as browser:
        browser.visit(url)
        time.sleep(1)
        for item in bs4(browser.html, 'lxml').find_all('div', class_='item'):
            product_item = item.find('a', class_='product-item') 
            title = product_item.img['alt'].replace(' Enhanced thumbnail', '')
            href = product_item['href']
            hemisphere_image_urls.append(dict(
                title=title,
                image_url=base_url + bs4(requests.get(
                    f'{base_url}{href}').text, 'lxml'
                             ).find('img', class_='wide-image')['src']
            ))
            
    return hemisphere_image_urls

In [77]:
get_mars_hemispheres()

[{'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'image_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'image_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere'},
 {'image_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere'}]

## Step 2 - MongoDB and Flask Application

In [78]:
def get_mission_to_mars_info():
    (news_title, news_p) = get_mars_news()
    return dict(
        news_title=news_title,
        news_p=news_p,
        featured_image_url=get_feature_image_url(),
        mars_weather=get_mars_weather(),
        mars_fact_html=get_mars_fact_html(),
        hemisphere_image_urls=get_mars_hemispheres(),
    )

In [79]:
get_mission_to_mars_info()

{'featured_image_url': 'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA18840_hires.jpg',
 'hemisphere_image_urls': [{'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
   'title': 'Cerberus Hemisphere'},
  {'image_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
   'title': 'Schiaparelli Hemisphere'},
  {'image_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
   'title': 'Syrtis Major Hemisphere'},
  {'image_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
   'title': 'Valles Marineris Hemisphere'}],
 'mars_fact_html': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measure</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  